<a href="https://colab.research.google.com/github/yeshwanthreddyp/BI-GRU-ATTENTION/blob/main/prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pydub

import os
from multiprocessing import Pool

import pandas as pd
import numpy as np

from sklearn.metrics import f1_score
from scipy.sparse import lil_matrix

import tensorflow as tf
from tqdm import tqdm

import librosa
from pydub import AudioSegment
from pydub.utils import make_chunks
from IPython.display import Audio

In [2]:
#assigning numerical class labels to each bird species

cls_label = {'Asiankoel' : 0, 'bluejay': 1, 'crow': 2, 'duck': 3, 'goaway': 4, 'lapwing': 5, 'owl': 6,
             'peafowl' : 7, 'sparrow':8, 'woodpeewe':9}

########################################################

def get_class_labels(r):
  x = r['class_name']
  label_names = x.split('_')

  label_nums=[]
  for l in label_names:
    indx = cls_label[l]
    label_nums.append(indx)

  return label_nums

In [5]:
# creating the dataframe which has path to wav file and it's label
# 2 species test dataset
label = []
path = []

root = '/content/drive/MyDrive/Colab Notebooks/whole_dataset/mix_2'
for species in os.listdir(root):
    species_folder = root + '/' + species
    for audio in os.listdir(species_folder):
        audio_path = species_folder + '/' + audio
        path.append(audio_path)
        label.append(species)

df_mix_2 = pd.DataFrame({'path': path,
                         'class_name': label})

df_mix_2['class_label'] = df_mix_2.apply(get_class_labels, axis=1)

print(df_mix_2.shape)
df_mix_2.sample(5)

(334, 3)


,path,class_name,class_label
36,/content/drive/MyDrive/Colab Notebooks/whole_d...,woodpeewe_lapwing,"[9, 5]"
56,/content/drive/MyDrive/Colab Notebooks/whole_d...,woodpeewe_owl,"[9, 6]"
269,/content/drive/MyDrive/Colab Notebooks/whole_d...,duck_bluejay,"[3, 1]"
228,/content/drive/MyDrive/Colab Notebooks/whole_d...,lapwing_goaway,"[5, 4]"
282,/content/drive/MyDrive/Colab Notebooks/whole_d...,duck_crow,"[3, 2]"


In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
# 3 species test dataset
label = []
path = []

root = '/content/drive/MyDrive/Colab Notebooks/whole_dataset/mix_3'
for species in os.listdir(root):
    species_folder = root + '/' + species
    for audio in os.listdir(species_folder):
        audio_path = species_folder + '/' + audio
        path.append(audio_path)
        label.append(species)

df_mix_3 = pd.DataFrame({'path': path,
                         'class_name': label})

df_mix_3['class_label'] = df_mix_3.apply(get_class_labels, axis=1)

print(df_mix_3.shape)
df_mix_3.sample(5)

(100, 3)


,path,class_name,class_label
69,/content/drive/MyDrive/Colab Notebooks/whole_d...,crow_sparrow_Asiankoel,"[2, 8, 0]"
71,/content/drive/MyDrive/Colab Notebooks/whole_d...,bluejay_woodpeewe_crow,"[1, 9, 2]"
96,/content/drive/MyDrive/Colab Notebooks/whole_d...,bluejay_peafowl_goaway,"[1, 7, 4]"
34,/content/drive/MyDrive/Colab Notebooks/whole_d...,duck_sparrow_bluejay,"[3, 8, 1]"
66,/content/drive/MyDrive/Colab Notebooks/whole_d...,crow_owl_goaway,"[2, 6, 4]"


In [7]:
# whole test data i.e mixture of both 2 and 3 species audio files
df = pd.concat([df_mix_2, df_mix_3], ignore_index = True)
df.sample(5)

,path,class_name,class_label
221,/content/drive/MyDrive/Colab Notebooks/whole_d...,lapwing_goaway,"[5, 4]"
333,/content/drive/MyDrive/Colab Notebooks/whole_d...,Asiankoel_duck,"[0, 3]"
81,/content/drive/MyDrive/Colab Notebooks/whole_d...,woodpeewe_peafowl,"[9, 7]"
246,/content/drive/MyDrive/Colab Notebooks/whole_d...,owl_sparrow,"[6, 8]"
381,/content/drive/MyDrive/Colab Notebooks/whole_d...,duck_lapwing_bluejay,"[3, 5, 1]"


In [8]:
# ref: https://www.kaggle.com/code/arailly/multi-label-f-beta-score/notebook

def label_to_sm(labels, n_classes):
    sm = lil_matrix((len(labels), n_classes))
    for i, label in enumerate(labels):
        sm[i, label] = 1  #eg: sm[0, [2,3]] = 1
    return sm

###############################################

def get_f1_score(y_true, y_pred):
  y_true_sm = label_to_sm(labels=y_true, n_classes=10)
  y_pred_sm = label_to_sm(labels=y_pred, n_classes=10)
  metric = f1_score(y_true=y_true_sm, y_pred=y_pred_sm, average='weighted')
  return metric


Prediction

In [10]:
path = '/content/drive/MyDrive/bi_gru_att_model'
#loading the model
model = tf.keras.models.load_model(path)

In [11]:
def get_split(path):
  myaudio = AudioSegment.from_file(path , "wav")
  chunk_length_ms = 1500 # pydub calculates in millisec
  chunks = make_chunks(myaudio, chunk_length_ms) #Make chunks of one sec

  #Export all of the individual chunks as wav files
  paths =[]

  for i, chunk in enumerate(chunks):
      chunk_name = "chunk{0}.wav".format(i)
      paths.append(chunk_name)
      chunk.export(chunk_name, format="wav")

  return paths

##################################################

def get_mfcc(path):
    y, sr = librosa.load(path)
    S = librosa.feature.melspectrogram(y=y, sr=sr)
    mfccs = librosa.feature.mfcc(S=librosa.power_to_db(S), n_mfcc=40)
    mfccs = mfccs.T
    return np.resize(mfccs, (65, 40))

##################################################

def get_xqs(paths):
  xqs = []
  for i in range(6): #only using first 6 chunks
    t = get_mfcc(paths[i])
    xqs.append(t)
  return xqs


In [12]:
def get_probs(model, xqs):
  probs = []
  for xq in xqs:
    xq = xq.reshape((1,65,40))
    p = model.predict(xq, verbose = 0)
    probs.append(p)
  return np.array(probs)

def predict(model, xqs, num_species):
  probs = get_probs(model, xqs)
  # aggregrate
  s = np.sum(probs, axis = 0)
  if num_species == 2:
    n = 2
  else:
    n = 3
  labels = np.argsort(s[0])[::-1][:n]
  return list(labels)

In [13]:
def test(model, df, num_species):
  y_true = list(df['class_label'])
  y_pred = []
  for i in tqdm(range(len(df))):
    path = df['path'][i]
    chunk_paths = get_split(path)
    xqs = get_xqs(chunk_paths)
    label = predict(model, xqs, num_species)
    y_pred.append(label)
  return y_true, y_pred

#########################################################

def evaluate(model, df_mix_2, df_mix_3, score):
  y_true_2, y_pred_2 = test(model, df_mix_2, 2)
  y_true_3, y_pred_3 = test(model, df_mix_3, 3)

  y_true = y_true_2 + y_true_3
  y_pred = y_pred_2 + y_pred_3

  if score:
    return get_f1_score(y_true, y_pred)
  else:
    return y_true, y_pred

In [14]:
evaluate(model, df_mix_2, df_mix_3, score = True)

100%|██████████| 100/100 [01:39<00:00,  1.01it/s]


0.7221660182028737

In [15]:
def get_sm(y_true, y_pred):
  y_true_sm = label_to_sm(labels=y_true, n_classes=10)
  y_pred_sm = label_to_sm(labels=y_pred, n_classes=10)
  return y_true_sm, y_pred_sm

##################################################################

y_true, y_pred = evaluate(model, df_mix_2, df_mix_3, score = False)
y_true, y_pred = get_sm(y_true, y_pred)

100%|██████████| 100/100 [01:01<00:00,  1.63it/s]


In [16]:
from sklearn.metrics import classification_report

print(classification_report(y_true, y_pred))

              precision    recall  f1-score   support

           0       0.80      0.97      0.88        69
           1       0.72      0.69      0.71       101
           2       0.74      0.92      0.82       100
           3       1.00      0.64      0.78        88
           4       0.52      1.00      0.68       145
           5       0.83      0.82      0.83       136
           6       1.00      0.35      0.51        55
           7       0.93      0.79      0.85        63
           8       0.65      0.26      0.37       119
           9       0.97      0.77      0.86        92

   micro avg       0.74      0.74      0.74       968
   macro avg       0.82      0.72      0.73       968
weighted avg       0.78      0.74      0.72       968
 samples avg       0.74      0.74      0.74       968



Demo

In [17]:
cls_label = {'Asiankoel' : 0, 'bluejay': 1, 'crow': 2, 'duck': 3, 'goaway': 4, 'lapwing': 5, 'owl': 6,
             'peafowl' : 7, 'sparrow':8, 'woodpeewe':9}

def get_bird_name(label):
    cls_names = []
    for k, v in cls_label.items():
      if v in label:
        cls_names.append(k)
    return cls_names

def demo(model, path, num_species):
    chunk_paths = get_split(path)
    xqs = get_xqs(chunk_paths)
    y_pred = predict(model, xqs, num_species)
    y_labels = get_bird_name(y_pred)
    return y_labels

In [18]:
i = 38
path = df_mix_2.iloc[i,0]
path

'/content/drive/MyDrive/Colab Notebooks/whole_dataset/mix_2/woodpeewe_lapwing/lapwing_woodpeewe_ (216).wav'

In [19]:
demo(model, path, 2)

['lapwing', 'woodpeewe']